In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch.cuda.empty_cache()

model_dir = "./Baichuan2-13B-Chat-v2.0.1"  # 克隆的仓库路径
model_name = "Baichuan2-13B-Chat"  # 模型名称
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # 自动分配 GPU/CPU
    trust_remote_code=True,
    torch_dtype="auto",  # 自动选择数据类型
    low_cpu_mem_usage=True  # 减少 CPU 内存使用
).eval()  # 切换到推理模式



/home/xuliren/anaconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.62s/it]


In [4]:
# 2. 构建符合模型要求的对话格式
import time
prompt = """你是一位资深质量工程师，请详细介绍：
1. PFMEA的定义
2. 核心三要素
3. 典型实施步骤
4. 汽车行业的应用案例"""

# 3. 生成优化配置
generate_kwargs = {
    "max_new_tokens": 512,
    "do_sample": False,    # 关闭随机采样保证确定性结果
    "temperature": 0.7,     # 可调低至0.3-0.5提升专业性
    "top_p": 0.9,
    "repetition_penalty": 1.2,  # 避免重复
    "pad_token_id": tokenizer.eos_token_id
}

# 4. 执行推理（首次运行会有编译延迟）
start = time.time()
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, **generate_kwargs)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"生成内容：\n{response}")
print(f"耗时：{time.time() - start:.2f}秒")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


生成内容：
你是一位资深质量工程师，请详细介绍：
1. PFMEA的定义
2. 核心三要素
3. 典型实施步骤
4. 汽车行业的应用案例及挑战分析。
耗时：2.28秒


In [2]:
# Helper function to run inference
def ask_model(prompt, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
prompt2 = """
你是一位资深质量工程师,请介绍一下什么是PFMEA？
"""

prompt1 = """
你是一位资深质量工程师，请根据以下质检数据生成一份完整的 PMEA 报告，内容包括问题描述、原因分析、控制对策建议等。

质检数据：
产品型号：AXZ-1023
缺陷：焊接不牢，导致漏液
发现频率：每批10%
分析：原材料合金波动 + 焊接设备老化

请以专业、清晰的语气输出报告。
"""
start = time.time()
response = ask_model(prompt1, max_new_tokens=1024)
print(f"使用时间 {time.time() - start:.2f}秒")
print("\n=== 模型输出 ===\n")
print(response)

使用时间 27.06秒

=== 模型输出 ===


你是一位资深质量工程师，请根据以下质检数据生成一份完整的 PMEA 报告，内容包括问题描述、原因分析、控制对策建议等。

质检数据：
产品型号：AXZ-1023
缺陷：焊接不牢，导致漏液
发现频率：每批10%
分析：原材料合金波动 + 焊接设备老化

请以专业、清晰的语气输出报告。

---

AXZ-1023产品质量改进报告

一、问题描述

近期在产品质量检查过程中发现，AXZ-1023型号产品的焊接部分存在焊接不牢的问题，导致产品在使用过程中出现漏液现象。该问题的发现在每个批次中占比约为10%，属于较为严重的质量问题。

二、原因分析

通过对质检数据的深入分析，我们认为该问题产生的原因主要包括以下几点：

1. 原材料合金波动：由于原材料合金成分的波动，可能导致焊接过程中的熔接效果不佳，从而影响焊接强度。

2. 焊接设备老化：长期使用导致焊接设备磨损老化，可能影响焊接过程的稳定性，进而降低焊接质量。

三、控制对策建议

针对上述原因，我们提出以下质量控制对策建议：

1. 对原材料进行更加严格的质量控制，确保合金成分稳定可靠，以降低因原材料问题导致的质量问题。

2. 对焊接设备进行全面检查和维护，及时更换老化部件，以确保焊接过程的稳定性和可靠性。

3. 加强生产线上的质量管理，对焊接环节进行定期抽查和监控，确保焊接质量得到有效控制。

4. 对员工进行焊接技能培训，提高操作技能水平，确保焊接过程严格按照工艺要求进行。

通过以上措施的实施，我们期望能够有效地解决AXZ-1023型号产品焊接不牢的问题，从而提高产品质量和客户满意度。


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [26]:
!nvidia-smi  # 查看 GPU 使用情况

Mon Jun 23 21:17:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0 Off |                  Off |
|  0%   48C    P8             15W /  450W |   13297MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from vllm import LLM, SamplingParams
model_path = "./Baichuan2-13B-Chat-v2.0.1"  # 模型名称
llm = LLM(model=model_path, 
          trust_remote_code=True,
          tensor_parallel_size=2,  # 设置张量并行度
          )
params = SamplingParams(
    max_tokens=512,
    temperature=0.7,
    top_p=0.9)

INFO 06-23 21:35:30 [config.py:823] This model supports multiple tasks: {'generate', 'classify', 'score', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 06-23 21:35:30 [config.py:1946] Defaulting to use mp for distributed inference
INFO 06-23 21:35:30 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 06-23 21:35:31 [tokenizer.py:262] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 06-23 21:35:31 [core.py:455] Waiting for init message from front-end.
INFO 06-23 21:35:31 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='./Baichuan2-13B-Chat-v2.0.1', speculative_config=None, tokenizer='./Baichuan2-13B-Chat-v2.0.1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipel

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


(VllmWorker rank=1 pid=73588) INFO 06-23 21:36:04 [default_loader.py:272] Loading weights took 7.71 seconds
(VllmWorker rank=0 pid=73587) INFO 06-23 21:36:05 [default_loader.py:272] Loading weights took 8.73 seconds
(VllmWorker rank=1 pid=73588) INFO 06-23 21:36:05 [gpu_model_runner.py:1624] Model loading took 13.0312 GiB and 7.883787 seconds
(VllmWorker rank=0 pid=73587) INFO 06-23 21:36:06 [gpu_model_runner.py:1624] Model loading took 13.0312 GiB and 8.886178 seconds
(VllmWorker rank=1 pid=73588) INFO 06-23 21:36:11 [backends.py:462] Using cache directory: /home/xuliren/.cache/vllm/torch_compile_cache/74d6036712/rank_1_0 for vLLM's torch.compile
(VllmWorker rank=1 pid=73588) INFO 06-23 21:36:11 [backends.py:472] Dynamo bytecode transform time: 5.20 s
(VllmWorker rank=0 pid=73587) INFO 06-23 21:36:12 [backends.py:462] Using cache directory: /home/xuliren/.cache/vllm/torch_compile_cache/74d6036712/rank_0_0 for vLLM's torch.compile
(VllmWorker rank=0 pid=73587) INFO 06-23 21:36:12 [back

In [6]:
prompt1 = """
你是一位资深质量工程师，请告诉我一个想转行做汽车质量工程师的人一些专业的指导建议。



请以专业、清晰的语气输出报告。
"""
outputs = llm.generate(prompt1, sampling_params=params)
print("\n=== vLLM 输出 ===\n")
print(outputs[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


=== vLLM 输出 ===




报告标题：汽车质量工程师转行指导报告



报告正文：

尊敬的汽车质量工程师转行意向者，

首先，感谢您对汽车行业质量工程领域的兴趣。作为一位资深质量工程师，我很高兴为您提供一些专业的建议，以助您顺利实现职业转型。

1. **了解汽车行业质量工程**：在开始转型之前，深入了解汽车行业质量工程的定义、职责、工作环境和行业趋势是至关重要的。这将帮助您确立职业目标，并确保您的技能和经验与行业需求相匹配。

2. **评估个人技能**：回顾您的教育背景、工作经验和现有技能，评估它们与汽车质量工程师职位要求的匹配程度。识别需要额外学习和发展的领域。

3. **获取相关证书**：许多汽车行业雇主倾向于招聘具有专业资格证书的质量工程师。考虑参加ISO/IATF 16949等质量管理体系认证培训，以增强您的职业竞争力。

4. **强化专业领域知识**：汽车质量工程师通常需要具备工程知识、质量管理工具和方法、以及车辆测试和验证技能。参加相关课程或在线资源，不断更新和深化这些专业知识。

5. **建立行业联系**：参加行业会议、研讨会和工作坊，建立与行业专家和专业人员的联系。这将帮助您了解行业动态，并为未来可能的职业机会建立人脉网络。

6. **实习或兼职**：如果您已经具备一定的汽车质量工程经验，考虑在汽车行业寻求实习或兼职机会。这将有助于您获取实际工作经验，并了解行业工作流程。

7. **保持学习态度**：汽车行业是快速发展的领域，技术不断更新。保持学习的态度，紧跟行业趋势和新技术，这将有助于您在职业生涯中保持竞争力。

8. **准备简历和求职信**：编写一份针对汽车质量工程师职位的简历和求职信。突出您的关键技能、成就和与职位相关的经验。

9. **准备面试**：了解汽车行业常见的面试问题和技巧，以便在求职过程中自信地展示您的能力。

10. **考虑专业指导**：在转型过程中，寻求一位经验丰富的质量工程师作为导师，提供职业发展的建议和指导。

最后，请记住，成功转行需要时间和努力。保持积极的态度，并准备好面对挑战。祝您转型顺利，成为一位杰出的汽车质量工程师。

此致，
[您的姓名]
资深质量工程师


In [22]:
from rag.retriever import retrieve  

In [23]:
retrieve("FEMA是什么")

[[1.000257  1.0229094 1.037869  1.0961313 1.1068993]] [[44 43 42 51 46]]


[]

In [15]:
from rag.retriever import load_static_faiss
index, texts = load_static_faiss()
texts[5]

'购或服务处理，需求部门后补PR/CER&PO，在OA提交\nPR/CER时应附上紧急采购申请单附件。\n4.5.2 正常采购项目\n公司各部门应根据本部门工作计划及批准的预算合理提前安排本部门的采购事宜。\n4..5.2.1有年度采购合同\n年度采购合同的对象和内容：经常消耗的物料或维修服务， 应与供应商签订年度采购合同。\n4.5.2.1.1 年度采购合同的竞价和签订\n采购员按如下竞价原则操作并提交相关经理签字审批。\n1） 对于经常消耗的物料或维修服务类业务采购员按照4.5.3.3竞价原则对比谈判之后确认优选\n供应商，非最低价中标供应商应由相关领导签字后方可签订年度采购协议。\n2） 年度采购合同到期前，'